In [1]:
import login as l
from datetime import datetime
from breeze_connect import BreezeConnect

# Initialize SDK
breeze = BreezeConnect(api_key=l.api_key)

# Generate Session
breeze.generate_session(api_secret=l.secret_key,
                            session_token=l.session_key)

# Connect to websocket(it will connect to tick-by-tick data server)
breeze.ws_connect()

def on_ticks():
        #Ordebook & Order Status callback to receive ticks
        if('sourceNumber' in ticks.keys()):
            status = ticks['orderStatus']
            order_id = ticks['orderReference']
            action = ticks['orderFlow']
            name = ticks['stockCode'] + ' | ' + ticks['expiryDate'] + ' | ' + ticks['strikePrice'] + ' | ' + ticks['optionType']
            
            print(f"ID : {order_id}\nAction : {action} {name}\nStatus : {status}\n")     #prints only order list

# Assign the callbacks.
breeze.on_ticks = on_ticks

breeze.subscribe_feeds(get_order_notification=True)  

{'message': 'Order Notification subscribed successfully'}

In [2]:
breeze.get_funds()

{'Success': {'bank_account': '39478995811',
  'total_bank_balance': 3227.0099999999998,
  'allocated_equity': 0.43,
  'allocated_fno': 3226.58,
  'block_by_trade_equity': 0.0,
  'block_by_trade_fno': 0.0,
  'block_by_trade_balance': 0.0,
  'unallocated_balance': '0'},
 'Status': 200,
 'Error': None}

In [3]:
def get_spot_price(stock):
    try:
        spot = breeze.get_quotes(stock_code=stock, exchange_code="NSE")['Success'][0]['ltp']
        print(f"SPOT: {spot}")
        return spot
    except Exception as e:
        print(f"API Error\n{e}\n")

In [5]:
spotprice = get_spot_price('NIFTY')
spotprice

SPOT: 21826.85


21826.85

In [6]:
slprice =spotprice - 5
slprice

21821.85

In [7]:
def get_atm_strike(stock):
    try:
        spot = breeze.get_quotes(stock_code=stock, exchange_code="NSE")['Success'][0]['ltp'] #get spot value
        atm_strike = 50 * (round(spot / 50))    #Calculate nearest strike price
        print(f"SPOT: {spot} | ATM_STRIKE : {atm_strike}")
        return atm_strike
    except Exception as e:
        print(f"API Error\n{e}\n")

In [8]:
atmstrike = get_atm_strike('NIFTY')
atmstrike

SPOT: 21826.6 | ATM_STRIKE : 21850 | STOPLOSS : 21831.6


(21850, 21831.6)

In [9]:
def get_expiry_dates(STOCK, STRIKE):
    try:
        expiry_dates_response = breeze.get_option_chain_quotes(
                                    stock_code=STOCK.upper(),
                                    exchange_code="NFO",
                                    product_type="options",
                                    expiry_date="",
                                    right="call",
                                    strike_price=STRIKE)
        
        if expiry_dates_response['Status'] == 200:
            expiry_dates = [expiry_dates_response['Success'][i]['expiry_date'] for i in range(len(expiry_dates_response['Success']))]
            return expiry_dates  # get list of all expiry for given stock and strike
        else:
            print(f'API Error\n{expiry_dates_response}')
    except Exception as e:
        print(f'API Error\n{e}\n')


In [10]:
expiry_dates = get_expiry_dates('NIFTY', get_atm_strike('NIFTY'))
expiry_dates

SPOT: 21826.6 | ATM_STRIKE : 21850 | STOPLOSS : 21831.6
API Error
'Status'



In [11]:
current_expiry = expiry_dates[0]
current_expiry

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# def get_contract(name, action):
#     name = name.upper()
#     details = name.split('-')
#     details[-1] = 'call' if (details[-1] == 'CE') else 'put'
    
#     if (details[2].split("/")[1] in ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']):
#         expiry = datetime.strptime(details[2], "%d/%b/%Y")
#     else:
#         expiry = datetime.strptime(details[2], "%d/%m/%Y")
        
#     contract = {
#         'stock':details[0],
#         'strike':details[1],
#         'expiry':expiry.strftime('%Y-%m-%dT06:00:00.000Z'),
#         'expiry_date':expiry.strftime('%d-%m-%Y'),
#         'right': details[-1],
#         'name': name,
#         'action': action
#     }
#     return contract

In [ ]:
def get_contract(name, action):
    name = name.upper()
    details = name.split('-')
    
    # Print details for debugging
    print("Details:", details)
    
    if len(details) < 6:
        # Handle the case where details does not have enough elements
        print("Error: Insufficient elements in details list.")
        return None
    
    details[-1] = 'call' if (details[-1] == 'CE') else 'put'
    
    # Convert month abbreviation to lowercase before formatting the date
    month_abbr_lower = details[3].lower()
    
    # Ensure the year is included in the date string
    expiry_date_str = f"{details[2]}-{month_abbr_lower}-{details[4]}-{details[5]}"
    
    if details[2].count("/") == 2 and details[3] in ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']:
        expiry = datetime.strptime(expiry_date_str, "%d-%b-%Y")
    else:
        expiry = datetime.strptime(expiry_date_str, "%d-%m-%Y")
        
    contract = {
        'stock': details[0],
        'strike': details[1],
        'expiry': expiry.strftime('%Y-%m-%dT06:00:00.000Z'),
        'expiry_date': expiry.strftime('%d-%m-%Y'),
        'right': details[-1],
        'name': name,
        'action': action
    }
    return contract

In [ ]:
get_contract('NIFTY-21700-04/01/2024-CE','buy')

In [ ]:
contract = get_contract('NIFTY-' + str(atmstrike) + '-' + current_expiry, 'buy')
# print("Contract:", contract)
contract

In [ ]:
# Import date class from datetime module
from datetime import date

# Returns the current local date
today = date.today()
print("Today date is: ", today)


In [ ]:
from datetime import datetime

# Get the current date and format it as required
current_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S.000Z')


#CALL BUY
breeze.place_order(
    stock_code="NIFTY",
    exchange_code="NFO",
    product="options",
    action="buy",
    order_type="market",
    stoploss= "",
    quantity="50",
    price="",
    validity="day",
    validity_date= current_date,  # Set to the current date
    disclosed_quantity="0",
    expiry_date= current_expiry,
    right="call",
    strike_price= get_atm_strike('NIFTY')
)


In [ ]:
#PUT BUY
breeze.place_order(stock_code="NIFTY",
                    exchange_code="NFO",
                    product="options",
                    action="buy",
                    order_type="market",
                    stoploss="21750",
                    quantity="50",
                    price="",
                    validity="day",
                    validity_date="2023-12-28T06:00:00.000Z",
                    disclosed_quantity="0",
                    expiry_date="2023-12-28T06:00:00.000Z",
                    right="put",
                    strike_price="21750")

In [ ]:
#SUARE OFF CAL
breeze.square_off(exchange_code="NFO",
                    product="options",
                    stock_code="NIFTY",
                    expiry_date="2023-12-28T06:00:00.000Z",
                    right="call",
                    strike_price="21750",
                    action="sell",
                    order_type="market",
                    validity="day",
                    stoploss="21749",
                    quantity="50",
                    price="0",
                    validity_date="2023-12-28T06:00:00.000Z",
                    trade_password="",
                    disclosed_quantity="0")

In [ ]:
#SUARE OFF PUT
breeze.square_off(exchange_code="NFO",
                    product="options",
                    stock_code="NIFTY",
                    expiry_date=current_expiry,
                    right="put",
                    strike_price=atmstrike,
                    action="sell",
                    order_type="market",
                    validity="day",
                    stoploss="0",
                    quantity="50",
                    price="0",
                    validity_date="2023-12-28T06:00:00.000Z",
                    trade_password="",
                    disclosed_quantity="0")

In [ ]:
breeze.get_portfolio_positions()

In [ ]:
#trailing stoploss, p&l in chatgpt